In [12]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

tqdm.pandas()

MODEL_NAME = "all-MiniLM-L6-v2"
K = 5
# Data fields IDs
# dropped all in the list
ID_LABELS = ["dataset_id", "article_id", "id", "DOI", "url"]
# data columns included in the input batch
# dropped: issued, embedding, author, text
TRAIN_LABELS = [ 'title', 'segments', 'extension', 'abstract', 'publisher', 'copyright', 'issued_year', 'all_authors', 'categories']
ALL_FIELDS = [
    'article_id','text','extension','source','dataset_id','dataset_id_cited','type',
    'id','categories','abstract','DOI','publisher','title','URL','copyright',
    'issued_year','all_authors','y'
]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class Basement(SentenceTransformer):
    def __init__(self, initial_data: pd.DataFrame):

        super().__init__(MODEL_NAME, DEVICE)

        self.x = self.preprocess_data(initial_data)
        self.to(DEVICE)

        self.eval()
        # freeze all parameters
        for p in self.parameters():
            p.requires_grad = False

    def _meta_conditioner(self, row):
        combined_prompt = f"""
        Title: {row.get('title', '')}
        Abstract: {row.get('abstract', '')}
        Author: {row.get('all_authors', '')}
        Extension File Type: {row.get('extension', '')}
        Publisher: {row.get('publisher', '')}
        Category: {row.get('categories', '')}
        Issued Year: {row.get('issued_year', '')}
        Copyright: {row.get('copyright', '')}
        """
        meta_embedding = self.encode([combined_prompt], convert_to_numpy=True, output_value="sentence_embedding") # or token_embeddings
        text_embedding = row.get('segments', None)

        if isinstance(text_embedding, np.ndarray):
            if text_embedding.ndim == 1:
                text_embedding = np.expand_dims(text_embedding, axis=0)
            return np.vstack((meta_embedding, text_embedding))

        # fallback empty vector if segments missing
        return np.empty((1, self.get_sentence_embedding_dimension()), dtype=np.float32)

    def _segment_text(self, s: str, k: int = K) -> np.ndarray:
        if not isinstance(s, str) or not s:
            return np.zeros((k, self.get_sentence_embedding_dimension()), dtype=np.float32)

        n = len(s)
        idx = np.linspace(0, n, k + 1, dtype=int)
        chunks = [s[idx[i]:idx[i+1]] for i in range(k)]
        emb = self.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
        emb = np.asarray(emb, dtype=np.float32)
        return emb.reshape(k, -1)

    def preprocess_data(
        self,
        data: pd.DataFrame,
        k: int = K,
        train_labels: list[str] | None = None
    ) -> pd.DataFrame:
        """
        - Adds 'segments' ((k, d) embeddings) from 'text'
        - Explodes: all_authors, segments, categories
        - Drops duplicates by TRAIN_LABELS (excluding 'segments')
        - Fills: copyright/cats/issued_year (mode)
        """
        train_labels = train_labels or TRAIN_LABELS

        df = data.copy()

        # 1) create 'segments'
        df.loc[:, "segments"] = df["text"].apply(lambda x: self._segment_text(x, k))

        # 2) explode multi-valued columns
        for col in ("all_authors", "segments", "categories"):
            if col in df.columns:
                df = df.explode(col, ignore_index=True)

        # 3) drop duplicated rows (excluding 'segments')
        subset_cols = [c for c in train_labels if c in df.columns and c != "segments"]
        if subset_cols:
            df = df.drop_duplicates(subset=subset_cols, keep="first")

        # 4) fills / cleaning
        if "issued_year" in df.columns:
            mode_series = df["issued_year"].dropna().mode()
            most_freq_issued = mode_series.iloc[0] if not mode_series.empty else None
            if most_freq_issued is not None:
                df.loc[:, "issued_year"] = df["issued_year"].fillna(most_freq_issued)

        if "copyright" in df.columns:
            df.loc[:, "copyright"] = (
                df["copyright"].astype("string").str.strip().replace({"": "Unknown"})
            )

        if "categories" in df.columns:
            df.loc[:, "categories"] = df["categories"].astype("string").fillna("Unknown")

        df.loc[:, "inputs"] = df.apply(self._meta_conditioner, axis=1)

        return df

    def predict(self, inputs: dict):
        max_pred = None
        text = inputs.get("text")
        if not text:
            return None

        # Build query embedding: meta + K segment embeddings -> mean pool to (1, D)
        segment = self._segment_text(text)
        inputs = dict(inputs, segments=segment)
        input_emb = self._meta_conditioner(inputs)            # (S_q, D)
        if input_emb.ndim == 1:
            input_emb = input_emb.reshape(1, -1)
        q = input_emb.mean(axis=0, keepdims=True)             # (1, D)

        print(f"query pooled emb shape: {q.shape}")

        # Compare to each training group
        for grp, row in self.x.groupby(['article_id', 'dataset_id', 'source']):
            # Stack that group's inputs -> (S_g, D)
            train_sample = np.vstack(row['inputs'].values)
            if train_sample.ndim == 1:
                train_sample = train_sample.reshape(1, -1)

            # Option A: compare every training vector to query, then average
            avg = float(cos_sim(train_sample, q).mean())

            # (Option B, faster): centroid = train_sample.mean(0, keepdims=True)
            # avg = float(cos_sim(centroid, q).item())

            prev = max_pred['score'] if max_pred else None
            if (max_pred is None) or (avg > prev):
                max_pred = {
                    'article_id': grp[0],
                    'dataset_id': grp[1],
                    'type': grp[2],
                    'score': avg,
                }

        return max_pred


In [13]:
# load data
train_data = pd.read_parquet('/kaggle/input/make-data-count-data-preparation/train_dataset.parquet')
test_data = pd.read_parquet('/kaggle/input/make-data-count-data-preparation/test_dataset.parquet')

In [14]:
base = Basement(train_data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
prediction = []

for _, row in test_data.iterrows():
    pred = base.predict(row.to_dict())   # convert Series → dict
    if pred:  # skip empty predictions
        prediction.append(pred)

submission = pd.DataFrame.from_records(prediction)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

query pooled emb shape: (1, 384)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

query pooled emb shape: (1, 384)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

query pooled emb shape: (1, 384)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

query pooled emb shape: (1, 384)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

query pooled emb shape: (1, 384)


In [ ]:
submission = submission.sort_values(["article_id", "dataset_id", "type"]).reset_index(drop=True)

# Add row_index (row_id) column
submission['row_index'] = submission.index

# Save in required order
submission[['row_index', 'article_id', 'dataset_id', 'type']].to_csv('/kaggle/working/submission.csv', index=False)
print('data submitted!')

In [ ]:
submission